### Script Features:
- **Dynamic Logging**: Creates timestamped log files for each run
- **Error Handling**: Comprehensive try-except blocks with logging
- **Data Fetching**: Retrieves historical data from Yahoo Finance
- **Bulk Insertion**: Uses `executemany` for efficient database writes
- **Duplicate Prevention**: Implements `ON CONFLICT DO NOTHING`
- **Progress Tracking**: Real-time console output with timestamps

### Workflow:
1. Connects to PostgreSQL to fetch S&P 500 tickers and their added dates
2. For each ticker:
   - Fetches historical data from Yahoo Finance starting from `added_date`
   - Previews first 5 rows in console
   - Converts data to appropriate types (float for prices, int for volume)
   - Inserts data into `stockdata` table
3. Logs all activities and errors to a timestamped file
4. Provides console feedback on progress and results

### Input Requirements:
- A populated `spy_companies_a` table with ticker symbols and added dates
- Internet connection for Yahoo Finance API access
- Valid PostgreSQL database credentials

### Output:
- **Database**: Updated `stockdata` table with historical stock prices
- **Log File**: Detailed execution log (e.g., `stock_history_log_20250329_143022.txt`)
- **Console**: Real-time progress updates and data previews

In [ ]:
import psycopg2
import yfinance as yf
import logging
import pandas as pd
from datetime import datetime

# Database connection parameters
DB_PARAMS = {
    "dbname": "postgres",
    "user": "postgres",
    "password": "Aarav12#$",
    "host": "npatelnynj.ddns.net",
    "port": "5432"
}

# Generate timestamped log file
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
log_filename = f"stock_history_log_{timestamp}.txt"

# Configure logging
logging.basicConfig(
    filename=log_filename,
    level=logging.INFO,
    format="%(asctime)s: %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S"
)

logging.info("************ Stock History Insertion Script Started ************")

# Function to fetch tickers from the database
def get_tickers():
    try:
        conn = psycopg2.connect(**DB_PARAMS)
        cur = conn.cursor()
        cur.execute("SELECT ticker, added_date FROM spy_companies_a;")
        tickers = cur.fetchall()  # List of (ticker, added_date)
        conn.close()
        logging.info(f"Fetched {len(tickers)} tickers from database.")
        return tickers
    except Exception as e:
        logging.error(f"Failed to fetch tickers: {e}")
        return []

# Function to fetch stock history from Yahoo Finance
def fetch_stock_history(ticker, start_date):
    try:
        print(f"Fetching data for {ticker} from {start_date} to today...")
        stock_data = yf.download(ticker, start=start_date, end=datetime.today().strftime("%Y-%m-%d"), progress=False)

        if stock_data.empty:
            logging.warning(f"No data found for {ticker}")
            print(f"⚠️ No data found for {ticker}. Skipping.")
            return None

        # Reset index to move 'Date' from index to a column
        stock_data.reset_index(inplace=True)

        # Flatten MultiIndex columns (remove the 'Ticker' level, keep 'Price' as column names)
        stock_data.columns = [col[0] if col[0] == 'Date' else col[0] for col in stock_data.columns]

        # Print first 5 rows of the fetched data for preview
        print(f"\nFirst 5 rows of data for {ticker}:")
        print(stock_data.head())
        print("Columns:", stock_data.columns)

        # Convert DataFrame to a list of tuples for bulk insertion
        result = [
            (
                ticker,
                row['Date'],  # Keep as Timestamp, PostgreSQL handles it
                float(row['Open']) if pd.notna(row['Open']) else None,
                float(row['High']) if pd.notna(row['High']) else None,
                float(row['Low']) if pd.notna(row['Low']) else None,
                float(row['Close']) if pd.notna(row['Close']) else None,
                int(row['Volume']) if pd.notna(row['Volume']) else None
            ) 
            for _, row in stock_data.iterrows()
        ]
        
        logging.info(f"Fetched {len(result)} rows for {ticker}.")
        return result
    except Exception as e:
        logging.error(f"Failed to fetch data for {ticker}: {e}")
        return None

# Function to insert stock history into the database
def insert_stock_history(ticker, stock_data):
    if not stock_data:
        logging.error(f"No valid data to insert for {ticker}. Skipping.")
        return 0

    try:
        conn = psycopg2.connect(**DB_PARAMS)
        cur = conn.cursor()

        insert_query = """
        INSERT INTO stockdata (stock_name, trade_date, open_price, high_price, low_price, close_price, volume)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (stock_name, trade_date) DO NOTHING;  -- Prevent duplicates
        """

        # Insert the stock_data directly
        cur.executemany(insert_query, stock_data)
        conn.commit()

        inserted_count = cur.rowcount
        cur.close()
        conn.close()
        
        logging.info(f"Inserted {inserted_count} rows for {ticker}.")
        return inserted_count
    except Exception as e:
        logging.error(f"Failed to insert data for {ticker}: {e}")
        return 0

# Main function to process all tickers
def process_spy_stocks():
    tickers = get_tickers()
    if not tickers:
        logging.error("No tickers found. Exiting process.")
        print("❌ No tickers found. Exiting.")
        return

    logging.info("****************************************************************")
    logging.info("Insertion of the Stock History Started")
    logging.info("****************************************************************")
    logging.info(f"Total Number of Tickers: {len(tickers)}")
    logging.info("****************************************************************")

    print("Starting stock history insertion...")

    for idx, (ticker, added_date) in enumerate(tickers, start=1):
        start_time = datetime.now().strftime("%H:%M:%S")

        print(f"\n[{start_time}] Processing ({idx}/{len(tickers)}) - {ticker}...")
        logging.info(f"\nProcessing {ticker} (Ticker {idx}/{len(tickers)})")
        logging.info(f"Fetching data from {added_date} to {datetime.today().date()}")

        stock_data = fetch_stock_history(ticker, added_date)
        if not stock_data:
            logging.error(f"Skipping {ticker} due to missing or empty data.")
            continue

        print(f"📊 {ticker}: {len(stock_data)} rows fetched. Inserting into DB...")
        inserted_count = insert_stock_history(ticker, stock_data)

        if inserted_count > 0:
            logging.info(f"✅ {ticker}: Inserted {inserted_count} rows.")
            print(f"✅ {ticker}: Inserted {inserted_count} rows.")
        else:
            logging.error(f"❌ {ticker}: Failed to insert.")
            print(f"❌ {ticker}: Failed to insert.")

        logging.info("****************************************************************")

    print("\nStock history insertion completed.")
    logging.info("************ Stock History Insertion Script Completed ************")

# Run the script
process_spy_stocks()